<img src="https://i.imgur.com/SsfgzAq.png" width=50%/>

# <b> Extract, Transform and Load for CourseData.db </b>
___

## Table of Content:
[1. Create Import Tables](#1)
<br> [2. Extract data from CSV files, Transform, Load into Import Tables](#2)
<br> [3. Create Tables from ERD](#3)
<br> [4. Populate the ERD Tables from The Imported Data Tables](#4)
<br> [5. Empty out the Import Tables to reclaim storage space](#5)

## Preliminaries: Extensions, Imports, and Database Connections

For more information about the following module and syntax, see the following sources:
- [sqlite3](https://docs.python.org/2/library/sqlite3.html)
- [create a database python using sqlite3](https://datatofish.com/create-database-python-using-sqlite3/)

In [37]:
# import sqlite3
import sqlite3

# to use the module, you must first create a Connection object that represents the database
# Note: You can create a new database by changing the name within the quotes
conn = sqlite3.connect('CourseData.db')  

# Once you have a Connection, you can create a Cursor object and call its execute() method to perform SQL commands:
c = conn.cursor() 

<a id = "1"> <h2> 1. Create Import Tables </h2> </a>
___
Create three tables where we can load the data into:
* IMPORT_COURSES
* IMPORT_COURSE_MEETINGS
* IMPORT_COURSE_CATALOG

To be able to import raw source data from .CSV files into in step 2:
* course_Meetings.csv
* courses.csv
* CourseCatalog2017_2018.csv 
* CourseCatalog2018_2019.csv

In [38]:
conn = sqlite3.connect('CourseData.db')
c = conn.cursor() 

# Create table - IMPORT_Courses
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSES''')
c.execute('''CREATE TABLE IMPORT_COURSES(
    [term] TEXT,
    [crn] TEXT,
    [catalog_id] TEXT,
    [section] TEXT,
    [credits] TEXT,
    [title] TEXT,
    [meetings] TEXT,
    [timecodes] TEXT,
    [primary_instructor] TEXT,
    [cap] INT,
    [act] INT, 
    [rem] INT,
    [instructor_fname] TEXT,
    [instructor_lname] TEXT,
    [semester] TEXT,
    [year] TEXT)'''
         )
          
# Create table - IMPORT_COURSE_MEETINGS
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_MEETINGS''')
c.execute('''CREATE TABLE IMPORT_COURSE_MEETINGS (
    [crn] TEXT,
    [location] TEXT,
    [day] TEXT,
    [start] TEXT,
    [end] TEXT,
    [term] TEXT,
    [semester] TEXT,
    [year] TEXT,
    [BuildingName]    TEXT , 
    [BuildingCode] TEXT )'''
         )
        
# Create table - IMPORT_COURSE_CATALOG
c.execute('''DROP TABLE IF EXISTS IMPORT_COURSE_CATALOG''')
c.execute('''CREATE TABLE IMPORT_COURSE_CATALOG (
    [program_code] TEXT,
    [program_name] TEXT,
    [catalog_ID] TEXT,
    [course_title] TEXT,
    [credits] TEXT,
    [prereqs] TEXT,
    [coreqs] TEXT,
    [fees] TEXT,
    [attributes] TEXT,
    [description] TEXT,
    [CatalogYear] TEXT)'''
         )

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

<b> 1.2 Test if the IMPORT Tables Are Created </b>
<br> by printing out the columns for each created import table.

For more information about the following modules, click on the link:
* [pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/overview.html)
* [sqlalchemy](https://www.sqlalchemy.org/)

In [39]:
from IPython.display import display, Markdown, Latex
# import pandas - providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data
import pandas as pd

# import sqlalchemy - Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL
from sqlalchemy import create_engine

In [40]:
# Create a new Engine instance - create an in-memory-only SQLite database
engine = create_engine("sqlite:///CourseData.db")

# Create a list of the IMPORT Tables
dbtable = ['IMPORT_COURSES',
           'IMPORT_COURSE_CATALOG' , 
           'IMPORT_COURSE_MEETINGS']

# Iterate through the list of tables and print out each column within every table
for i in dbtable :
    display(Markdown('**Table :{}**'.format( i)))
    tables = pd.read_sql_table(i,engine)
    display(tables.head())

**Table :IMPORT_COURSES**

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,instructor_fname,instructor_lname,semester,year


**Table :IMPORT_COURSE_CATALOG**

,program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,CatalogYear


**Table :IMPORT_COURSE_MEETINGS**

,crn,location,day,start,end,term,semester,year,BuildingName,BuildingCode


<a id = "2"> <h2> 2. Extract data from CSV files, Transform, Load into Import Tables </h2> </a>
___
Located in the subfolders of the folder 'SourceData' are the following CSV files: 
* courses.csv*
* course_meetings.csv*
* CourseCatalog2017_2018.csv 
* CourseCatalog2018_2019.csv
<br> <em> *whereby courses.csv and course_meetings.csv are 2 files per semester (for 19 semesters, between the years: 2014-2018) </em>

Steps:
1. First we extract the data from each CSV file and convert it to a pandas dataframe (for easier manipulation using Python syntax)
2. Then we impute 'missing'/incorrect data for each dataframe
3. Lastly we put the cleaned data from the imputed dataframe into the corresponding IMPORT tables:
    * courses.csv -> df_courses -> IMPORT_COURSES
    * course_meetings.csv** -> df_course_meetings_buildings -> IMPORT_COURSE_CATALOG
    * CourseCatalog2017_2018.csv -> data -> IMPORT_COURSE_MEETINGS
    * CourseCatalog2018_2019.csv -> data -> IMPORT_COURSE_MEETINGS
    
<em> **Note: we add external data for the buildings based on 'Location', namely the 'BuildingName' and 'BuildingCode' </em>

<b> 2.1 Create a Dataframe for courses.csv files combined from all subfolders </b>
<br> for the syntax the following source was used: [link](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe)

In [41]:
# create a list of subfolder names located in the 'SourceData' folder for the semesters 
semester_folders = ['Fall2014',
                    'Fall2015',
                    'Fall2016',
                    'Fall2017',
                    'Fall2018',
                    'Spring2015', 
                    'Spring2016',
                    'Spring2017',
                    'Spring2018',
                    'Spring2019',
                    'SpringBreak2017',
                    'Summer2015',
                    'Summer2016',
                    'Summer2017',
                    'Summer2018',
                    'Winter2015',
                    'Winter2016', 
                    'Winter2017',
                    'Winter2018']

# --------------------------------------------------
# loop through the files and read them in with pandas
# --------------------------------------------------
# create an empty list to hold all the individual pandas dataframes
dataframes = []  

# Iterate through the list of semester_folders 
for semester in semester_folders:
    # the path to your csv file directory
    fixing_catalog_years
    mycsvdir = 'SourceData/{}/courses.csv'.format(semester)
    # read all the csv files in that directory (assuming they have the extension courses.csv)
    df = pd.read_csv(mycsvdir)
    #  append rows for each dataframe to the end of the given list
    dataframes.append(df)

# concatenate them all together
result = pd.concat(dataframes, ignore_index=True)
# print out to a new csv file
result.to_csv('courses_all.csv')
display(Markdown('**Number of rows: {0} <br> Number of columns: {1}**'.format(result.shape[0],result.shape[1])))

**Number of rows: 15937 <br> Number of columns: 12**

<b> 2.2 Impute the dataframe representing the combined courses files: 'courses_all.csv' </b>
- trim whitespaces for all columns
- split strings and create new columns for:
    * column 'primary_instructor' into two columns: first name, last name
    * column 'term' into: semester, year
- replace empty brackets [] for the 'meetings' column
- replace empty brackets [] for the 'timecodes' column
- replace 'missing' values for the primary instructor (which are dates) with 'TBA' (which stands for: To Be Announced)
- replace 'instructor_lname' column's NaN's with 'TBA'

<b> Load in the CSV file of the combined courses.csv files as a pandas dataframe

In [42]:
df_courses = pd.read_csv('courses_all.csv', header=0, index_col=0)

* <b> Primary Instructor: </b> find the unique values for primary instructor that don't have names but dates in them
<br> _Note:_ originally found wrong values by checking a 'courses_all.csv' file CSV file in Excel

In [43]:
# import python package re - for regular expression operations
import re 

# define a list of unique values that start with a integer between 0 and 9 in the column 'primary_instructor'
primary_instructor_values = list(df_courses[df_courses['primary_instructor'].str.contains(r'[0-9]+') == True]['primary_instructor'].unique())

# show the list of wrong unique values to replace
display(Markdown('**Unique values to replace:** {}'.format( len(primary_instructor_values))))
for i in primary_instructor_values:
    display(Markdown(i))

**Unique values to replace:** 8

09/15-10/31

09/02-12/30

08/15-01/15

09/02-12/19

09/01-12/07

01/20-05/08

01/20-04/30

05/21-08/17

* <b> Meetings: </b> find the number of values that have brackets [] instead of actual meetings that are needed to replace
<br> _Note:_ originally found wrong values by checking a 'courses_all.csv' file CSV file in Excel

In [44]:


len_col_meetings_miss = df_courses['meetings'].value_counts()[0]
len_col_meetings =  df_courses['meetings'].size
display(Markdown("Number of values that have `[]` in the `meetings` column: **{0}**".format(len_col_meetings_miss)))
display(Markdown('**% Meetings missing that need to be replaced: {:0.2f}%**'.format(len_col_meetings_miss/len_col_meetings*100)  ))


Number of values that have `[]` in the `meetings` column: **2491**

**% Meetings missing that need to be replaced: 15.63%**

* <b> Timecodes: </b> find the number of values that have brackets [] instead of actual timecodes that are needed to replace
<br> _Note:_ originally found wrong values by checking a 'courses_all.csv' file CSV file in Excel

In [45]:
len_col_timecodes_miss = df_courses[df_courses['timecodes'] == '[]']['timecodes'].size
len_col_timecodes = df_courses['timecodes'].size


display(Markdown("Number of values that have `[]` in the `timecodes` column: **{}**".format(len_col_timecodes_miss)))
display(Markdown("**% Timecodes missing that need to be replaced: {:0.2f} %**".format(len_col_timecodes_miss/len_col_timecodes*100)   ))

Number of values that have `[]` in the `timecodes` column: **2491**

**% Timecodes missing that need to be replaced: 15.63 %**

---

<b> Define two functions for: </b>
- trim whitespaces (beginning/end)
- remove spaces 
<br> For the syntax the following source was used, see [link](https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe)

In [46]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

import pandas as pd

def remove_whitespace(x):
    try:
        # remove spaces inside and outside of string
        x = "".join(x.split())

    except:
        pass
    return x

* Impute values in 'df_courses' pandas dataframe

In [47]:
# --------------------------------------------------------------------------------------------
# Apply to remove_whitespace to entire Dataframe
# --------------------------------------------------------------------------------------------
# apply the function to trim whitespaces
df_courses = trim_all_columns(df_courses)

# --------------------------------------------------------------------------------------------
# split the first and last name of the instructor from the 'primary_instructor' column 
# by the pattern: 2 spaces '  ', example 'John J.  Doe' would become 'John J.' and 'Doe'
# --------------------------------------------------------------------------------------------
df_courses[['instructor_fname','instructor_lname']] = df_courses['primary_instructor'].str.split('  ',expand=True,)

# --------------------------------------------------------------------------------------------
# split the semester and year form the 'term' column by the first integer '2', 
# example: Fall2014 will be split into 'Fall' and '2014'
# --------------------------------------------------------------------------------------------
df_courses[['semester','year']] = df_courses['term'].str.split('2',expand=True,)
# for column 'year' just created, add back the integer 2 to to year for each row (this 2 was used to split on and was subtracted)
df_courses['year'] = '2'+df_courses['year']

# --------------------------------------------------------------------------------------------
# impute meetings and timecodes columns
# --------------------------------------------------------------------------------------------
df_courses['meetings'].loc[df_courses.meetings == '[]'] = 'No meetings'
df_courses['timecodes'].loc[df_courses.timecodes == '[]']= 'No time codes'
# --------------------------------------------------------------------------------------------

# cleaning the missing values in primary_instructor
# --------------------------------------------------------------------------------------------
for i in primary_instructor_values :
    #print(df_courses.primary_instructor.loc[df_courses.primary_instructor == i].count())# = 'No meetings'
    df_courses['primary_instructor'].loc[df_courses.primary_instructor == i] = 'TBA'
    
# --------------------------------------------------------------------------------------------
# replace all the NaN's for the column 'instructor_lname' to 'TBA'    
# --------------------------------------------------------------------------------------------    
df_courses["instructor_lname"] = df_courses["instructor_lname"].fillna("TBA")

<b> 2.3 Test if the following new columns were created correctly:
- 'instructor_fname' 
- 'instructor_lname'

In [48]:
# show the first 2 rows of the dataframe (transposed)
display(df_courses.head(2).T)

# show the descriptives of the numeric columns of the dataframe (transposed)
display(df_courses.describe().T)

,0,1
term,Fall2014,Fall2014
crn,70384,70385
catalog_id,AC 0011,AC 0011
section,C01,C02
credits,3.000,3.000
title,Introduction to Financial Accounting,Introduction to Financial Accounting
meetings,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...","[{'days': 'TF', 'times': '0930am-1045am', 'dat..."
timecodes,['TF 0800am-0915am 09/02-12/08 DSB 105'],['TF 0930am-1045am 09/02-12/08 DSB 105']
primary_instructor,Michael P. Coyne,Michael P. Coyne
cap,0,0


,count,mean,std,min,25%,50%,75%,max
crn,15937.0,55172.299178,19308.879968,10893.0,35426.0,57145.0,74541.0,79215.0
cap,15937.0,17.992784,12.671906,0.0,10.0,19.0,25.0,500.0
act,15937.0,15.578591,10.107690,-1.0,7.0,15.0,23.0,90.0
rem,15937.0,2.477317,12.709781,-75.0,0.0,1.0,5.0,999.0


<b> 2.4 Test if there are any remaining NaN's in 'df_courses'

In [49]:
df_courses.isna().sum()

term                  0
crn                   0
catalog_id            0
section               0
credits               0
title                 0
meetings              0
timecodes             0
primary_instructor    0
cap                   0
act                   0
rem                   0
instructor_fname      0
instructor_lname      0
semester              0
year                  0
dtype: int64

<b> 2.5 Add the imputed dataframe 'df_courses' to the SQLITE IMPORT table: 'IMPORT_COURSES' </b>

In [50]:
# create a Connection object that represents the database
# You can create a new database by changing the name within the quotes
conn = sqlite3.connect('CourseData.db')  

# Once you have a Connection, you can create a Cursor object and call its execute() method to perform SQL commands:
c = conn.cursor()

# append the pandas dataframe 'df_courses' to the IMPORT_COURSES SQLITE Table
df_courses.to_sql('IMPORT_COURSES', conn, if_exists='append', index=False)

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

<b> 2.6 Create a Dataframe for course_meetings.csv files combined from all subfolders </b>
<br> the following source was used as a reference for the syntax: [link](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe)

In [51]:
# define the names of the subfolders, where to extract course_meetings.csv from:
semester_folders = ['Fall2014', 
                    'Fall2015',
                    'Fall2016',
                    'Fall2017',
                    'Fall2018',
                    'Spring2015', 
                    'Spring2016',
                    'Spring2017',
                    'Spring2018',
                    'Spring2019',
                    'SpringBreak2017',
                    'Summer2015',
                    'Summer2016',
                    'Summer2017',
                    'Summer2018',
                    'Winter2015',
                    'Winter2016', 
                    'Winter2017',
                    'Winter2018']

# loop through the files and read them in with pandas
# a list to hold all the individual pandas DataFrames
dataframes = []  
for semester in semester_folders:
    # the path to your csv file directory
    mycsvdir = 'SourceData/{}/course_meetings.csv'.format(semester)
    # read all the csv files in that directory (assuming they have the extension course_meetings.csv)
    df = pd.read_csv(mycsvdir)
    dataframes.append(df)

# concatenate the dataframes all together
result = pd.concat(dataframes, ignore_index=True)

# save the combined dataframes of course_meetings to a new csv file called: 'course_meetings_all'
result.to_csv('course_meetings_all.csv')

In [52]:
# create a dataframe for the course_meetings_all CSV file
df_course_meetings = pd.read_csv('course_meetings_all.csv' ,header=0, index_col=0)

# apply function to trim all columns
df_course_meetings = trim_all_columns(df_course_meetings)

# split the 'term' into two new columns 'semester' and 'year' and split on the '2' 
df_course_meetings[['semester','year']] = df_course_meetings['term'].str.split('2',expand=True,)

# add back the 2 in the year, which was the integer which was split on
df_course_meetings['year'] = '2'+df_course_meetings['year']

# print the shape of the dataframe (rows, columns)
print(df_course_meetings.shape)

# read in the 'buildings2.csv' file with the columns: location, BuildingName, BuildingCode
df_b = pd.read_csv('buildings2.csv' ,header=0, index_col=0)

# apply function to trim the columns of the buildings2.csv file
df_b = trim_all_columns(df_b)

# combine the two dataframes: df_course_meetings and df_b which contains the: location, BuildinName and BuildingCode
df_course_meetings_buildings = df_course_meetings.merge(df_b  ,left_on='location', right_on='location')

# show the shape of the new combined dataframe, to see if the two columns where added based on the 'location' column
print(df_course_meetings_buildings.shape)

(284907, 8)
(284907, 10)


In [53]:
# show the first 5 rows of the combined dataframe to visually inspect if the BuildingName and BuildingCode were added
df_course_meetings_buildings.head()

,term,crn,location,day,start,end,semester,year,BuildingName,BuildingCode
0,Fall2014,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00,Fall,2014,Dolan School of Business,DSB
1,Fall2014,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00,Fall,2014,Dolan School of Business,DSB
2,Fall2014,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00,Fall,2014,Dolan School of Business,DSB
3,Fall2014,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00,Fall,2014,Dolan School of Business,DSB
4,Fall2014,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00,Fall,2014,Dolan School of Business,DSB


<b> 2.7 Add the imputed dataframe 'df_course_meetings_buildings' to the SQLITE IMPORT table: 'IMPORT_COURSES' </b>

In [54]:
# create a Connection object that represents the database
# You can create a new database by changing the name within the quotes
conn = sqlite3.connect('CourseData.db')

# Once you have a Connection, you can create a Cursor object and call its execute() method to perform SQL commands:
c = conn.cursor() 

# append the pandas dataframe 'df_course_meetings_buildings' to the 'IMPORT_COURSE_MEETINGS' SQLITE Table
df_course_meetings_buildings.to_sql('IMPORT_COURSE_MEETINGS', conn, if_exists='append', index=False)

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

<b> 2.8 Add the two CSV files of the Catalogs for 2018-2019 and 2017-2018 to the IMPORT table: 'IMPORT_COURSE_CATALOG' </b>
<br> The syntax was based on the code provided by Prof. Christopher Huntley, see [link](https://github.com/christopherhuntley/ba510-course-data-project/blob/example/ETL_CourseData.ipynb)

In [55]:
# create a Connection object that represents the database
# You can create a new database by changing the name within the quotes
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes

# Once you have a Connection, you can create a Cursor object and call its execute() method to perform SQL commands:
c = conn.cursor()

# define the suffix of the CSV files for: 'CourseCatalog2018_2019.csv' and CourseCatalog2017_2018.csv
catalog_years = ['2017_2018', '2018_2019']

# ----------------------------------------------------------------
# iterate through list 'catalog_years' and do the following:
# ----------------------------------------------------------------
# 1. define the filepath
# 2. read in the csv file as a pandas dataframe
# 3. trim all columns in the dataframe
# 4. remove whitespaces in the dataframe
# 5. add a new column CatalogYear in the dataframe
# 6. Add the dataframe to the sqlite table 'IMPORT_COURSE_CATALOG'
# ----------------------------------------------------------------
for cat_year in catalog_years:
    # define the dynamic filepath which will be loaded with an element from the list 'catalog_years'
    filepath = 'SourceData/Catalogs/CourseCatalog{}.csv'.format(cat_year)
    # convert the csv file to a pandas dataframe
    data = pd.read_csv(filepath)
    # apply fucntion to trim all the columns
    data = trim_all_columns(data)
    # apply function to remove whitespaces for the column 'program_code'
    data['program_code'] = data['program_code'].apply(remove_whitespace)
    data['CatalogYear'] = cat_year
    data.to_sql('IMPORT_COURSE_CATALOG',conn,if_exists='append',index=False)

# We close the connection of the 'CourseData.db' file when we are done with it.
conn.close()


# create a Connection object that represents the database
# You can create a new database by changing the name within the quotes
conn = sqlite3.connect('CourseData.db')  # You can create a new database by changing the name within the quotes

# Once you have a Connection, you can create a Cursor object and call its execute() method to perform SQL commands:
c = conn.cursor()

# define the suffix of the CSV files for: 'CourseCatalog2018_2019_clean2.csv' and CourseCatalog2017_2018,etc..

catalog_years = ['2015_2016' ,'2016_2017' ,'2017_2018' ,"2018_2019"]

for cat_year in catalog_years:
     # define the dynamic filepath which will be loaded with an element from the list 'catalog_years'

    filepath = 'fixing_catalog_years/CourseCatalog{}_clean2.csv'.format(cat_year)
    
    # convert the csv file to a pandas dataframe

    data = pd.read_csv(filepath , skipinitialspace=True,encoding = 'unicode_escape')
   
    data= data.drop(['Unnamed: 0.1.1','Unnamed: 0','Unnamed: 0.1' ,'YEAR' , "SEMESTER" ,"term"], axis=1, errors='ignore')
    data.rename(columns = { 'Title' : 'course_title' }, inplace = True)
    data.rename(columns = { 'catalog_id' : 'catalog_ID' }, inplace = True)
    # apply fucntion to trim all the columns
    data = trim_all_columns(data)
    
    data['CatalogYear'] = cat_year
    data.to_sql('IMPORT_COURSE_CATALOG',conn,if_exists='append',index=False)
    
    

   
conn.close()

<b> 2.9 Test if the data from the CSV files is loaded into the following IMPORT tables: </b>
* IMPORT_COURSES
* IMPORT_COURSE_CATALOG
* IMPORT_COURSE_MEETINGS

In [56]:
# create a Connection object that represents the database
engine = create_engine("sqlite:///CourseData.db")

# define the list with the names of the SQL IMPORT tables
dbtable = ['IMPORT_COURSES' ,'IMPORT_COURSE_CATALOG' , 'IMPORT_COURSE_MEETINGS']

# Iterate through the list of tables and display the first 5 rows for each IMPORT table
for i in dbtable :
    # print out the Table name as a reference to the user above the dataframe
    display(Markdown('**Table :{}**'.format( i)))
    # convert the SQL tables to a pandas dataframe
    tables = pd.read_sql_table(i,engine)
    # display the first 5 rows of the IMPORT TABLE
    display(tables.head())
    display(Markdown("---"))

**Table :IMPORT_COURSES**

,term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,instructor_fname,instructor_lname,semester,year
0,Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dat...",['TF 0800am-0915am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31,Michael P.,Coyne,Fall,2014
1,Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dat...",['TF 0930am-1045am 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31,Michael P.,Coyne,Fall,2014
2,Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dat...",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],Michael P. Coyne,0,31,-31,Michael P.,Coyne,Fall,2014
3,Fall2014,70291,AC 0011,C04,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1100am-1215pm', 'dat...",['MR 1100am-1215pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,29,-29,Rebecca I.,Bloch,Fall,2014
4,Fall2014,70350,AC 0011,C05,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dat...",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],Rebecca I. Bloch,0,30,-30,Rebecca I.,Bloch,Fall,2014


---

**Table :IMPORT_COURSE_CATALOG**

,program_code,program_name,catalog_ID,course_title,credits,prereqs,coreqs,fees,attributes,description,CatalogYear
0,AN,Asian Studies,AN 0301,Independent Study,1-3 Credits,None,None,None,None,Students undertake an individualized program o...,2017_2018
1,AN,Asian Studies,AN 0310,Asian Studies Seminar,3 Credits,None,None,None,None,This seminar examines selected topics concerni...,2017_2018
2,BU,Business,BU 0211,Legal Environment of Business,3 Credits,Junior standing.,None,None,None,This course examines the broad philosophical a...,2017_2018
3,BU,Business,BU 0220,Environmental Law and Policy,3 Credits,None,None,None,"EVME Environmental Studies Major Elective, EVP...",This course surveys issues arising out of fede...,2017_2018
4,BU,Business,BU 0311,"The Law of Contracts, Sales, and Property",3 Credits,BU 0211.,None,None,None,This course examines the components of common ...,2017_2018


---

**Table :IMPORT_COURSE_MEETINGS**

,crn,location,day,start,end,term,semester,year,BuildingName,BuildingCode
0,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
1,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
2,70384,DSB 105,T,2014-09-09T08:00:00,2014-09-09T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
3,70384,DSB 105,F,2014-09-12T08:00:00,2014-09-12T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB
4,70384,DSB 105,T,2014-09-16T08:00:00,2014-09-16T09:15:00,Fall2014,Fall,2014,Dolan School of Business,DSB


---

<a id = "3"> <h2> 3. Create Tables from ERD </h2> </a>
___
To reference the ERD, please click this [link](docs/CourseDataERDv1.pdf)

Create the following SQLITE tables:
* PROGRAMS
* FACULTY
* LOCATION
* COURSE
* COURSE_OFFERING
* CLASS_MEETINGS
* CATALOG_YEAR

In [57]:
# Create a new Engine instance - create an in-memory-only SQLite database
# You can create a new database by changing the name within the quotes
conn = sqlite3.connect('CourseData.db')  

# Once you have a Connection, you can create a Cursor object and call its execute() method to perform SQL commands:
c = conn.cursor()

# Create table - PROGRAMS
c.execute('''DROP TABLE IF EXISTS PROGRAMS''')
c.execute('''CREATE TABLE PROGRAMS ( 
    [PID] INTEGER PRIMARY KEY,
    [program_code] TEXT NOT NULL,
    [program_name] TEXT NOT NULL)''')

# Create table - FACULTY
c.execute('''DROP TABLE IF EXISTS FACULTY''')
c.execute('''CREATE TABLE FACULTY ( 
    [FID] INTEGER PRIMARY KEY,
    [instructor_fname] TEXT NOT NULL,
    [instructor_lname] TEXT NOT NULL)''')

# Create table - LOCATION
c.execute('''DROP TABLE IF EXISTS LOCATION''')
c.execute('''CREATE TABLE LOCATION ( 
    [LID] INTEGER PRIMARY KEY,
    [BuildingName] TEXT,
    [BuildingCode] TEXT,
    [location] TEXT)''')

# Create table - COURSE
c.execute('''DROP TABLE IF EXISTS COURSE''')
c.execute('''CREATE TABLE COURSE ( 
    [CID] INTEGER PRIMARY KEY,
    [PID] INTEGER NOT NULL,
    [CatalogYear] TEXT NOT NULL,
    [catalog_ID] TEXT NOT NULL,
    [course_title] TEXT NOT NULL,
    [credits] TEXT NOT NULL,
    [prereqs] TEXT,
    [coreqs] TEXT,
    [fees] TEXT,
    [attributes] TEXT,
    [description] TEXT,
    FOREIGN KEY (PID) REFERENCES PROGRAMS (PID)
)''')

# Create table - COURSE_OFFERING
c.execute('''DROP TABLE IF EXISTS COURSE_OFFERING''')
c.execute('''CREATE TABLE COURSE_OFFERING ( 
    [COID] INTEGER PRIMARY KEY,
    [CID] INTEGER,
    [FID] INTEGER NOT NULL,
    [CRN] TEXT NOT NULL,
    [SEMESTER] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    [TITLE] TEXT NOT NULL,
    [CATALOG_ID] TEXT,
    [CatalogYear] TEXT,
    [MEETINGS] TEXT,
    [TIMECODES] TEXT,
    [SECTION] TEXT NOT NULL,
    [CAP] INTEGER NOT NULL,
    [ACT] INTEGER NOT NULL,
    [REM] INTEGER NOT NULL,
    [instructor_lname] TEXT,
    [instructor_fname] TEXT,
    FOREIGN KEY (CID) REFERENCES COURSE (CID)
    FOREIGN KEY (FID) REFERENCES FACULTY (FID)
)''')

# Create table - CLASS_MEETINGS
c.execute('''DROP TABLE IF EXISTS CLASS_MEETINGS''')
c.execute('''CREATE TABLE CLASS_MEETINGS ( 
    [CMID] INTEGER PRIMARY KEY,
    [COID] INTEGER,
    [LID] INTEGER,
    [DAY] TEXT,
    [START] TEXT,
    [END] TEXT,
    [CRN] TEXT,
    [SEMESTER] TEXT NOT NULL,
    [YEAR] INTEGER NOT NULL,
    FOREIGN KEY (LID) REFERENCES LOCATION (LID)
    FOREIGN KEY (COID) REFERENCES COURSE_OFFERING (COID)
)''')

# Create table - CATALOG_YEAR
c.execute('''DROP TABLE IF EXISTS CATALOG_YEAR''')
c.execute('''CREATE TABLE CATALOG_YEAR (
    [CatalogYear] TEXT,
    [YEAR] INTERGER ,
    [SEMESTER] TEXT
)''')
         
# Save (commit) the changes    
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

<a id = "4"> <h2> 4. Populate the ERD Tables from The Imported Data Tables </h2> </a>
___

Populate the following SQLITE tables in this order:
1. CATALOG_YEAR
2. PROGRAMS
3. FACULTY
4. LOCATION
5. COURSE
6. COURSE_OFFERING
7. CLASS_MEETINGS

In [58]:
# Connect to the database 'CourseData'
conn = sqlite3.connect('CourseData.db')

# Once you have a Connection, you can create a Cursor object and call its execute() method to perform SQL commands:
c = conn.cursor()

#  INSERT Data Into Table: 'CATALOG_YEAR' for 3 columns: Semester,Year and CatalogYear 
c.execute('''INSERT INTO CATALOG_YEAR (Semester,Year,CatalogYear) VALUES 
('Fall','2014','2014_2015'),('Winter','2015','2014_2015'),
('Spring','2015','2014_2015'),('Summer','2015','2014_2015'),
('Fall','2015','2015_2016'),('Winter','2016','2015_2016'),
('Spring','2016','2015_2016'),('Summer','2016','2015_2016'),
('Fall','2016','2016_2017'),('Winter','2017','2016_2017'),
('Spring','2017','2016_2017'),('SpringBreak','2017','2016_2017'),
('Summer','2017','2016_2017'),('Fall','2017','2017_2018'),
('Winter','2018','2017_2018'),('Spring','2018','2017_2018'),
('Summer','2018','2017_2018'),('Fall','2018','2018_2019'),
('Winter','2019','2018_2019'),('Spring','2019','2018_2019')'''
         )

# INSERT Data into Table: 'PROGRAMS' FROM 'IMPORT_COURSE_CATALOG'
c.execute('''INSERT INTO PROGRAMS (program_code,program_name) 
SELECT DISTINCT program_code,program_name
FROM IMPORT_COURSE_CATALOG''')

# INSERT Data into Table: 'FACULTY' for 2 columns: first names and last names of instructors FROM table 'IMPORT_COURSES'
c.execute('''INSERT INTO FACULTY (instructor_lname, instructor_fname)
SELECT DISTINCT instructor_lname, instructor_fname
FROM IMPORT_COURSES''')

# INSERT Data into Table: 'LOCATION' FROM 'IMPORT_COURSE_MEETINGS'
c.execute('''INSERT INTO LOCATION (BuildingName, BuildingCode, Location) 
SELECT DISTINCT BuildingName,BuildingCode,location
FROM IMPORT_COURSE_MEETINGS''')

# INSERT Data into Table: 'COURSE' FROM 'IMPORT_COURSE_CATALOG'
c.execute('''INSERT INTO COURSE (PID, CatalogYear, Catalog_ID, Course_Title, Credits, Prereqs, Coreqs, Fees, Attributes, Description)
SELECT DISTINCT PID,CatalogYear,Catalog_ID,Course_Title,Credits,Prereqs,Coreqs,Fees,Attributes,Description
FROM IMPORT_COURSE_CATALOG
     JOIN PROGRAMS ON (PROGRAMS.program_code=IMPORT_COURSE_CATALOG.program_code)''')

# INSERT Data into Table: 'COURSE_OFFERING'
c.execute('''INSERT INTO COURSE_OFFERING (CID, FID, CRN, Semester, Year, Title, Catalog_ID, CatalogYear, Meetings, Timecodes, Section, Cap, Act, Rem, instructor_lname, instructor_fname)
SELECT DISTINCT CID, FID, CRN, Semester, Year, Title, catalog_ID, CatalogYear, Meetings, Timecodes, Section, Cap, Act, Rem, instructor_lname, instructor_fname
FROM IMPORT_COURSES
    LEFT JOIN CATALOG_YEAR using(semester, year)
    LEFT JOIN COURSE using( CatalogYear, catalog_ID)
    LEFT JOIN FACULTY using (instructor_lname, instructor_fname)''')

# INSERT Data into Table: 'CLASS_MEETINGS'
c.execute('''INSERT INTO CLASS_MEETINGS (COID, LID, CRN, Day, Start, End, SEMESTER, YEAR)
SELECT DISTINCT COURSE_OFFERING.COID, LOCATION.LID, COURSE_OFFERING.CRN, Day, Start, End, COURSE_OFFERING.SEMESTER, COURSE_OFFERING.YEAR
FROM IMPORT_COURSE_MEETINGS
    JOIN COURSE_OFFERING USING (SEMESTER, YEAR, CRN)
    LEFT JOIN LOCATION ON (IMPORT_COURSE_MEETINGS.Location = LOCATION.location)''')

# Save (commit) the changes              
conn.commit()

# Close the connection
# Just be sure any changes have been committed or they will be lost.
conn.close()

<a id = "5"> <h2> 5. Empty out the IMPORT Tables to reclaim storage space </h2> </a>
___    

<b> 6.1 Drop IMPORT tables in the database: </b>
- IMPORT_COURSE_MEETINGS
- IMPORT_COURSES
- IMPORT_COURSE_CATALOG
<br> _Note:_ We no longer need these tables, because the data is already imported into our ERD Tables.

In [59]:
#%load_ext sql
%reload_ext sql

# initialize a database connection
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

In [60]:
%%sql
-- Delete all data
DELETE FROM IMPORT_COURSE_MEETINGS;
DELETE FROM IMPORT_COURSES;
DELETE FROM IMPORT_COURSE_CATALOG;

 * sqlite:///CourseData.db
Done.
15937 rows affected.
6775 rows affected.


[]

In [61]:
%%sql
-- Drop the tables
DROP TABLE IMPORT_COURSE_MEETINGS;
DROP TABLE IMPORT_COURSES;
DROP TABLE IMPORT_COURSE_CATALOG;

 * sqlite:///CourseData.db
Done.
Done.
Done.


[]

In [62]:
# show non-truncated long strings in pandas columns
pd.set_option('display.max_colwidth', -1)

#checking the db tables after dropping the import tables.
query = 'SELECT * from sqlite_master where type= "table"'
engine = create_engine("sqlite:///CourseData.db")

tables = pd.read_sql_query(query,engine)
tables = tables.rename(columns={"name": "DatabaseTables"})
tables

,type,DatabaseTables,tbl_name,rootpage,sql
0,table,PROGRAMS,PROGRAMS,2,"CREATE TABLE PROGRAMS ( \n [PID] INTEGER PRIMARY KEY,\n [program_code] TEXT NOT NULL,\n [program_name] TEXT NOT NULL)"
1,table,FACULTY,FACULTY,3,"CREATE TABLE FACULTY ( \n [FID] INTEGER PRIMARY KEY,\n [instructor_fname] TEXT NOT NULL,\n [instructor_lname] TEXT NOT NULL)"
2,table,LOCATION,LOCATION,4,"CREATE TABLE LOCATION ( \n [LID] INTEGER PRIMARY KEY,\n [BuildingName] TEXT,\n [BuildingCode] TEXT,\n [location] TEXT)"
3,table,COURSE,COURSE,5,"CREATE TABLE COURSE ( \n [CID] INTEGER PRIMARY KEY,\n [PID] INTEGER NOT NULL,\n [CatalogYear] TEXT NOT NULL,\n [catalog_ID] TEXT NOT NULL,\n [course_title] TEXT NOT NULL,\n [credits] TEXT NOT NULL,\n [prereqs] TEXT,\n [coreqs] TEXT,\n [fees] TEXT,\n [attributes] TEXT,\n [description] TEXT,\n FOREIGN KEY (PID) REFERENCES PROGRAMS (PID)\n)"
4,table,COURSE_OFFERING,COURSE_OFFERING,6,"CREATE TABLE COURSE_OFFERING ( \n [COID] INTEGER PRIMARY KEY,\n [CID] INTEGER,\n [FID] INTEGER NOT NULL,\n [CRN] TEXT NOT NULL,\n [SEMESTER] TEXT NOT NULL,\n [YEAR] INTEGER NOT NULL,\n [TITLE] TEXT NOT NULL,\n [CATALOG_ID] TEXT,\n [CatalogYear] TEXT,\n [MEETINGS] TEXT,\n [TIMECODES] TEXT,\n [SECTION] TEXT NOT NULL,\n [CAP] INTEGER NOT NULL,\n [ACT] INTEGER NOT NULL,\n [REM] INTEGER NOT NULL,\n [instructor_lname] TEXT,\n [instructor_fname] TEXT,\n FOREIGN KEY (CID) REFERENCES COURSE (CID)\n FOREIGN KEY (FID) REFERENCES FACULTY (FID)\n)"
5,table,CLASS_MEETINGS,CLASS_MEETINGS,7,"CREATE TABLE CLASS_MEETINGS ( \n [CMID] INTEGER PRIMARY KEY,\n [COID] INTEGER,\n [LID] INTEGER,\n [DAY] TEXT,\n [START] TEXT,\n [END] TEXT,\n [CRN] TEXT,\n [SEMESTER] TEXT NOT NULL,\n [YEAR] INTEGER NOT NULL,\n FOREIGN KEY (LID) REFERENCES LOCATION (LID)\n FOREIGN KEY (COID) REFERENCES COURSE_OFFERING (COID)\n)"
6,table,CATALOG_YEAR,CATALOG_YEAR,8,"CREATE TABLE CATALOG_YEAR (\n [CatalogYear] TEXT,\n [YEAR] INTERGER ,\n [SEMESTER] TEXT\n)"


In [63]:
%%sql
-- Force SQLite to rebuild the database file to minimize file size
vacuum;

 * sqlite:///CourseData.db
Done.


[]